# Preparation

In [ ]:
import os
workdir = os.path.expanduser('~/simulations/spinw')
!mkdir -p {workdir}
%cd {workdir}

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import sys
# sys.path.insert(0, '/SNS/users/lj7/dv/mcvine/dgsres/')
import dgsres
dgsres

In [ ]:
from dgsres.singlextal import workflow
reload(workflow)

In [ ]:
import imp
config = imp.load_source('config', '/path/to/convolution_config.py')

In [ ]:
for sl in config.slices:
    workflow.create_convolution_calculator(sl)

# Prepare image to convolve

In [ ]:
# Code to find out what is the image to calculate to be convoluted
import imp
config = imp.load_source('config', '/path/to/convolution_config.py')
from dgsres.singlextal import workflow
slices_of_interest = config.slices[:]
# Checking that the grid dimensions are ok
for slice_index in range(0,len(slices_of_interest)):
    #slice_index=0
    sl = config.slices[slice_index]
    workflow.create_convolution_calculator(sl) 
    convolver = sl.convolution.calculator
    print("Slice {0}:".format(sl.name))
    print(str(convolver.finer_expanded_grid))
    print("Array Shape of the model: {0}x{1}".format(len(convolver.finer_expanded_grid.xaxis.ticks()),len(convolver.finer_expanded_grid.yaxis.ticks())))
    print('-'*50+"\n")

# Convolve

In [ ]:
%%time
%matplotlib inline
for i, sl in enumerate(config.slices):
    print sl.name
    workflow.create_convolution_calculator(sl)
    convolver = sl.convolution.calculator
    img = np.load('./%s_spinw.npy' % sl.name)
    convolved = convolver.convolve(img.T)
    np.save('./%s_convolved.npy' % sl.name, convolved)    

    plt.figure(figsize=(5,5))
    qgrid, Egrid = np.meshgrid(
        convolver.finer_expanded_grid.xaxis.ticks(),
        convolver.finer_expanded_grid.yaxis.ticks(),
    )
    plt.pcolormesh(qgrid, Egrid, convolved)
    plt.xlabel('q')
    plt.colorbar()
    plt.ylabel('E (meV)')
    plt.savefig('%s_IqE.png' % sl.name)
    plt.close()

# Plot all

In [ ]:
%matplotlib notebook

In [ ]:
# plt.figure()
f, axarr = plt.subplots(1,4, sharex=False, sharey=True, figsize=(9.5, 6))

xticks_list = [
    range(4),
    range(3),
    range(3),
    range(3)
]
xlimits_list = [
    (0, 4), 
    (0,2),
    (2,0),
    (0,2.5)
]

for i, (sl, xticks, xlimits) in enumerate(zip(config.slices, xticks_list, xlimits_list)):
    # print sl.name
    img = np.load('./%s_convolved.npy' % sl.name)
    convolver = sl.convolution.calculator
    qgrid, Egrid = np.meshgrid(
        convolver.finer_expanded_grid.xaxis.ticks(),
        convolver.finer_expanded_grid.yaxis.ticks(),
    )
    ax = axarr[i]
    ax.pcolormesh(qgrid, Egrid, img, vmin=0, vmax=100) #, cmap='jet')
    ax.set_xticks(xticks)
    ax.set_xlim(*xlimits)
    ax.set_xlabel(sl.name)

# plt.xlabel('q')
plt.ylim(0, 25)
ax.set_ylabel('E(meV)')
# plt.xticks(np.arange(4))
# plt.colorbar()
# plt.ylabel('E (meV)')

f.subplots_adjust(wspace=0)
for ax in axarr:
    ax.label_outer()